**Dans toute la suite de ce cours, nous travaillerons sur la base de donnée suivante**

![](https://capytale2.ac-paris.fr/web/sites/default/files/2021-12-29-10-48-31/livres.png)

# Modification des données

Les données stockées dans un SGBD ne sont a priori pas figées et peuvent être modifiées au cours du temps. Nous allons montrer deux types de modifications pouvant être faites sur les tables :

- la suppression d'un ensemble de lignes et 
- la mise à jour de certains attributs d'un ensemble de lignes.


##  Suppression de lignes

L'ordre `DELETE FROM t WHERE c` permet de supprimer de la table `t` toutes les lignes vérifiant la condition `c`.

Dans l'exemple de notre médiathèque, supposons que l'utilisateur `Sébastien Petit`, dont le `code_barre` est `'934701281931582'`, ait rendu ses livres. Il faut supprimer de la table `emprunt` toutes les lignes pour lesquelles le `code_barre` vaut `'934701281931582'`, ce qui donne l'ordre suivant:

```sql
DELETE FROM emprunt
WHERE code_barre = '934701281931582';
```

Après exécution de cet ordre, la recherche dans la table emprunt ne donne plus de résultats :

```sql
SELECT COUNT(*) AS total
FROM emprunt
WHERE code_barre = '934701281931582';
```

alors qu'avant la suppression, cette requête envoyait un `total` de `2`.

In [1]:
SELECT COUNT(*) AS total
FROM emprunt
WHERE code_barre = '934701281931582';

total
2


#### Erreur classique

**Attention**, au même titre qu'une requête `SELECT` sans clause `WHERE` sélectionne **toutes les lignes**, un ordre `DELETE` sans clause `WHERE` **efface toutes les lignes** de la table.

Il ne faut pas confondre `DELETE FROM t` et `DROP TABLE t` :

- La première opération vide une table de son contenu, mais ne supprime pas la table. Il est donc possible d'y ajouter de nouveau des données au moyen de l'instruction INSERT.
- La seconde opération détruit la table (et ses données). La table ne peut donc plus être référencée.

#### Respect des contraintes

Comme nous l'avons dit précédemment, les **contraintes sont vérifiées à chaque mise à jour**. 

Essayons de supprimer le livre `Hacker's delight` de la table `livre`, sachant que l'ISBN de ce dernier est `'978-0201914658'`.

```sql
DELETE FROM livre WHERE isbn = '978-0201914658';
```

renvoit l'erreur suivante :

```
Error: FOREIGN KEY constraint failed
```

In [2]:
DELETE FROM livre WHERE isbn = '978-0201914658';

Error: FOREIGN KEY constraint failed

Error: 

Ici, le SGBD nous indique que supprimer ce livre (et donc supprimer sa clé primaire de la table livre) violerait la contrainte de clé étrangère dans la table `auteur_de`.

Comme pour la destruction d'une table, il faut donc supprimer :

- en premier les lignes dont les attributs sont déclarés comme **clés étrangères**
- avant de supprimer celles contenant les **clés primaires** correspondantes.

#### Atomicité d'une requête

Du point de vue de leur exécution, les ordres de modification de table sont 

- soit **entièrement exécutés**,
- soit **entièrement annulés**.

Considérons la requête suivante:

```sql
DELETE FROM usager WHERE cp = '75019' OR cp = '75020' ;
```

qui efface de la table usager toutes les personnes dont le code postal est `75019` ou `75020`.

- Si aucune de ces personnes n'apparaît dans la table emprunt, alors les suppressions peuvent être effectuées sans erreur.
- Supposons maintenant que certaines de ces personnes ont emprunté un livre. Même si le SGDB rencontre en premier des personnes sans emprunt et les supprime, il lèvera une erreur dès qu'il rencontrera un usager référencé dans la table `emprunt`. Dans ce cas :
    - toutes les modifications déjà faites seront annulées et
    - la table se trouvera dans l'état qu'elle avait avant la tentative d'exécution.
    
Ici, la requête sera donc complètement annulée et l'usager de code postal `75019` ne sera pas supprimé.


Les exécutions sont donc de type **tout ou rien**, ce que l'on appelle la propriété d'**atomicité**.

In [3]:
DELETE FROM usager WHERE cp = '75019' OR cp = '75020' ;

Error: FOREIGN KEY constraint failed

Error: 

## Mise à jour

Le second type de modification est la mise à jour. Elle consiste à remplacer certains attributs d'un ensemble de lignes par de nouvelles valeurs.

La syntaxe est la suivante :

    UPDATE t SET a1 = e1, SET a2 = e2, ..., SET an = en WHERE c
    
Cette dernière signifie *sélectionne dans la table `t` toutes les lignes vérifiant la condition `c` et, pour chacune de ces lignes, remplace la valeur courante de l'**attribut** `ai` par la **valeur** de l'expression `ei`*.


Par exemple, la requête suivante permet d'afficher l'adresse e-mail actuelle de l'utilisateur `Sébastien Petit` :

```sql
SELECT email from  usager
WHERE code_barre = '934701281931582';
```

et nous affiche

![](https://capytale2.ac-paris.fr/web/sites/default/files/2021-12-29-13-35-24/10.png)

Pour mettre à jour son adresse `email`, on écrira ceci:

```sql
UPDATE usager SET email = 'sebastien.petit@hmail.com'
WHERE code_barre = '934701281931582';
```

On peut vérifier avec la requête `SELECT` que l'adresse a bien été modifiée


![](https://capytale2.ac-paris.fr/web/sites/default/files/2021-12-29-13-35-24/11.png)

In [4]:
SELECT email from  usager
WHERE code_barre = '934701281931582';

email
spetit4@email.fr


In [5]:
UPDATE usager SET email = 'sebastien.petit@hmail.com'
WHERE code_barre = '934701281931582';

In [6]:
SELECT email from  usager
WHERE code_barre = '934701281931582';

email
sebastien.petit@hmail.com


Les expressions de mise à jour peuvent mentionner des noms d'attributs. Ces derniers sont alors remplacés par la valeur courante (avant mise à jour) de ces attributs.

Supposons par exemple que la médiathèque soit fermée au mois d'avril. On souhaite que tous les `emprunts` dont la date de rendu était en avril soient prolongés de 30 jours.

```sql
UPDATE emprunt SET retour = retour + 30
WHERE retour >= '2020-04-01';
```


#### Hors programme

Cette syntaxe `retour + 30` est très simple et fonctionne car `retour` a pour domaine `DATE`.

**Mais** en SQLite (et donc dans ce notebook), elle ne fonctionne pas car le type `DATE` n'existe pas ici... 
La syntaxe correcte, plus complexe et **hors programme** est la suivante 

```sql
SET retour = date(retour,'+30 day')
```

et utilise la fonction `date()` qui permet des opérations entres chaînes de caractères interprétées comme des dates.


In [7]:
SELECT * FROM emprunt
WHERE retour >= '2020-04-01';

code_barre,isbn,retour
421921003090881,978-2081358881,2020-04-28
421921003090881,978-2207249123,2020-04-28
421921003090881,978-2824709420,2020-04-28
917547585216771,978-2290105504,2020-04-07


In [8]:
UPDATE emprunt
-- début du commentaire
-- SET retour = retour + 30 
-- fin du commentaire
SET retour = date(retour,'+30 day')
WHERE retour >= '2020-04-01';

In [9]:
SELECT * FROM emprunt
WHERE retour >= '2020-04-01';

code_barre,isbn,retour
421921003090881,978-2081358881,2020-05-28
421921003090881,978-2207249123,2020-05-28
421921003090881,978-2824709420,2020-05-28
917547585216771,978-2290105504,2020-05-07


Dans la mise à jour précédente, la clause `SET retour = retour + 30` est similaire à la modification d'une variable dans un langage de programmation comme Python, c'est-à-dire :

1. prendre la valeur courante de retour,
2. y ajouter 30 et 
3. écrire la nouvelle valeur dans retour.

## Requêtes imbriquées

#### Première manière

Il est possible de créer une table de manière *temporaire* et d'exécuter une requête sur cette table en **imbriquant** la première requête dans la clause `FROM` de la seconde ou dans une clause `JOIN ... ON` :

```sql
SELECT * FROM (SELECT * FROM livre
                        WHERE annee >= 1990) AS tmp
WHERE tmp.annee <= 2000;
```

La requête ci-dessus calcule d'abord une table intermédiaire nommée `tmp` qui liste les livres publiés après 1990. Suite à quoi, la table `tmp` est refiltrée pour ne garder que les livres pour lesquels l'année est inférieure à 2000. Attention, il ne s'agit ici que d'une explication de *haut niveau*. En pratique, n'importe quel SGBD moderne évaluera cette requête imbriquée comme la requête équivalente :

```sql
SELECT * FROM livre WHERE annee >= 1990 AND annee <= 2000;
```

In [10]:
SELECT * FROM (SELECT * FROM livre
                        WHERE annee >= 1990) AS tmp
WHERE tmp.annee <= 2000;

titre,editeur,annee,isbn
Fondation et Empire,Editions Denoël,1999,978-2207249123
Akira,Glénat,2000,978-2723428262
Croisière sans escale,Editions Denoël,1990,978-2207500293
Astérix et Cléopâtre,Dargaud,1999,978-2012100060
Au carrefour des étoiles,J'ai Lu,1997,978-2277118473
Les Buddenbrook,LGF/Le Livre de Poche,1993,978-2253063193
Le Noir Dessein,Livre de poche,1998,978-2253062820
The Practice of Programming,Addison-Wesley Professional,1999,978-0201615869
Les Plus qu'humains,adsaa,1999,000-0000000162
Les Contes de Canterbury,Gallimard Education,2000,978-2070406340


#### Deuxième manière

Une autre manière d'imbriquer les requêtes consiste à utiliser une sous-requête dans la clause `WHERE`.

En effet, le langage SQL identifie les valeurs scalaires et les tables à une seule *case* telles que celles renvoyées par les fonctions d'agrégation. Par exemple, si on souhaite afficher les titres des livres dont l'année est la plus ancienne dans la base, on pourra écrire:

```sql
SELECT titre FROM livre WHERE annee =
                        (SELECT MIN(annee) FROM livre);
```

Ici, la sous-requête calcule l'année minimum de la table `livre` (*1933* dans notre base), puis affiche tous les titres de livres dont l'année vaut `1933`.

In [11]:
SELECT titre FROM livre WHERE annee =
                        (SELECT MIN(annee) FROM livre);

titre
Berlin Alexanderplatz


Attention, la sous-requête ne doit pas nécessairement comporter une fonction d'agrégation. Il suffit qu'elle renvoie une table contenant **une seule valeur**.

Ainsi, si nous voulons afficher les titres des livres publiés la même année que Moby Dick (sans connaître cette année), nous pouvons écrire :

```sql
SELECT titre FROM livre WHERE annee =
                    (SELECT annee FROM livre WHERE titre = 'Moby Dick');
```

In [12]:
SELECT titre FROM livre WHERE annee =
                    (SELECT annee FROM livre WHERE titre = 'Moby Dick');

titre
Moby Dick
L'Île des morts
Le Devin
Le Berceau du chat
Les Enfants de minuit
À la recherche du temps perdu


**Attention** si la sous-requête renvoie plusieurs résultats, le SGBD renverra une erreur :

```sql
SELECT titre FROM livre WHERE annee =
            (SELECT annee FROM livre WHERE titre LIKE '%Astérix%');
```

(sauf dans ce notebook et en SQLite, mais on va faire comme si...)